In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten 
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam

In [ ]:
import tensorflow.keras.backend as K

def f2(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    f2 = 5*p*r / (4*p+r+K.epsilon())
    return K.mean(f2)

def f2_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)
    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())
    f2 = 5*p*r / (4*p+r+K.epsilon())
    return 1 - K.mean(f2)

In [ ]:
#import efficientnet.tfkeras
from tensorflow.keras.models import load_model

model_line = load_model('model_line_6515.h5', custom_objects={'f2_loss': f2_loss,"f2": f2})
model_rampRU = load_model("model_rampRU_6515.h5", custom_objects={'f2_loss': f2_loss,"f2": f2})
model_rampRD = load_model("model_rampRD_6515.h5", custom_objects={'f2_loss': f2_loss,"f2": f2})

In [ ]:
import scipy.io as io

import hdf5storage
InputDBY = hdf5storage.loadmat('InputDBY_6515.mat')
InputDBY=InputDBY['InputDBY']

SimulationData = hdf5storage.loadmat('SimulationData_6515.mat')
LastIterline=SimulationData['LastIterline']
LastIterRU=SimulationData['LastIterRU']
LastIterRD=SimulationData['LastIterRD']

InputDBY=np.transpose(InputDBY, (3,0,1,2))
LastIterline=np.transpose(LastIterline, (2,0,1))
LastIterRU=np.transpose(LastIterRU, (2,0,1))
LastIterRD=np.transpose(LastIterRD, (2,0,1))

InputDBY=np.float32(InputDBY)
LastIterline=np.float32(LastIterline)
LastIterRU=np.float32(LastIterRU)
LastIterRD=np.float32(LastIterRD)

print(InputDBY.shape)
print(LastIterline.shape)
print(LastIterRU.shape)
print(LastIterRD.shape)

InputDBY=InputDBY[0:3710,0:,0:,0:]
LastIterline=LastIterline[0:3710,0:]
LastIterRU=LastIterRU[0:3710,0:]
LastIterRD=LastIterRD[0:3710,0:]

print(InputDBY.shape)
print(LastIterline.shape)
print(LastIterRU.shape)
print(LastIterRD.shape)

Nunits=1388
Nbus=6515
Nbranch=9037
Horizon=12

n_scenario=3710
n_train=2710
n_test=1000

In [ ]:
#Flattening target data
LastIterline=np.reshape(LastIterline,(n_scenario,Nbranch*12),'F')
LastIterRU=np.reshape(LastIterRU,(n_scenario,Nunits*11),'F')
LastIterRD=np.reshape(LastIterRD,(n_scenario,Nunits*11),'F')

print(LastIterline.shape)
print(LastIterRU.shape)
print(LastIterRD.shape)

In [ ]:
#dataset for testing
x_test=InputDBY[n_train:,0:,0:,0:]
y_test_line=LastIterline[n_train:,0:]
y_test_rampRU=LastIterRU[n_train:,0:]
y_test_rampRD=LastIterRD[n_train:,0:]

print(x_test.shape)
print(y_test_line.shape)
print(y_test_rampRU.shape)
print(y_test_rampRD.shape)

#normalizing testing data
#finding the stat of testing data
maxD=np.amax(x_test[0:,0:,0:,0])
maxC=np.amax(x_test[0:,0:,0:,1])
#maxT=np.amax(x_test[0:,0:,0:,2])

x_test[0:,0:,0:,0]=x_test[0:,0:,0:,0]/maxD
x_test[0:,0:,0:,1]=x_test[0:,0:,0:,1]/maxC
#x_test[0:,0:,0:,2]=x_test[0:,0:,0:,2]/maxT

In [ ]:
# prediction 
y_pred_line=model_line.predict(x_test)
y_pred_rampRU=model_rampRU.predict(x_test)
y_pred_rampRD=model_rampRD.predict(x_test)
print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

In [ ]:
from sklearn.metrics import confusion_matrix
probability_fitering=np.array([0.999])

#testing the Branch classifier
y_test_line=np.reshape(y_test_line,np.size(y_test_line,0)*np.size(y_test_line,1),'c')
y_pred_line=np.reshape(y_pred_line,np.size(y_pred_line,0)*np.size(y_pred_line,1),'c')

y_pred_line[y_pred_line >= probability_fitering] = 1
y_pred_line[y_pred_line < probability_fitering] = 0
print(confusion_matrix(y_test_line, y_pred_line))

#testing the Ramp Up classifier
y_test_rampRU=np.reshape(y_test_rampRU,np.size(y_test_rampRU,0)*np.size(y_test_rampRU,1),'c')
y_pred_rampRU=np.reshape(y_pred_rampRU,np.size(y_pred_rampRU,0)*np.size(y_pred_rampRU,1),'c')

y_pred_rampRU[y_pred_rampRU >= probability_fitering] = 1
y_pred_rampRU[y_pred_rampRU < probability_fitering] = 0
print(confusion_matrix(y_test_rampRU, y_pred_rampRU))

#testing the Ramp Down classifier
y_test_rampRD=np.reshape(y_test_rampRD,np.size(y_test_rampRD,0)*np.size(y_test_rampRD,1),'c')
y_pred_rampRD=np.reshape(y_pred_rampRD,np.size(y_pred_rampRD,0)*np.size(y_pred_rampRD,1),'c')

y_pred_rampRD[y_pred_rampRD >= probability_fitering] = 1
y_pred_rampRD[y_pred_rampRD < probability_fitering] = 0
print(confusion_matrix(y_test_rampRD, y_pred_rampRD))

print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

In [ ]:
#ouput data reshaping
y_pred_line=np.reshape(y_pred_line,(n_test,Nbranch*12),'c')
y_pred_rampRU=np.reshape(y_pred_rampRU,(n_test,Nunits*11),'c')
y_pred_rampRD=np.reshape(y_pred_rampRD,(n_test,Nunits*11),'c')
print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

#Breaking each scenario into row and columns
y_pred_line=np.reshape(y_pred_line,(n_test,Nbranch,12),'F')
y_pred_rampRU=np.reshape(y_pred_rampRU,(n_test,Nunits,11),'F')
y_pred_rampRD=np.reshape(y_pred_rampRD,(n_test,Nunits,11),'F')
print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

y_pred_line=np.transpose(y_pred_line, (1,2,0))
y_pred_rampRU=np.transpose(y_pred_rampRU, (1,2,0))
y_pred_rampRD=np.transpose(y_pred_rampRD, (1,2,0))
print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

In [ ]:
#saving the predicted result back to image format
predicted_6515 = {"predicted_branch": y_pred_line,"predicted_rampRU": y_pred_rampRU,"predicted_rampRD": y_pred_rampRD}
io.savemat(r'predicted_6515_DCT.mat', predicted_6515)

In [ ]:
#dataset for testing
x_test=InputDBY[n_train:,0:,0:,0:]
y_test_line=LastIterline[n_train:,0:]
y_test_rampRU=LastIterRU[n_train:,0:]
y_test_rampRD=LastIterRD[n_train:,0:]

print(x_test.shape)
print(y_test_line.shape)
print(y_test_rampRU.shape)
print(y_test_rampRD.shape)

#normalizing testing data
#finding the stat of testing data
maxD=np.amax(x_test[0:,0:,0:,0])
maxC=np.amax(x_test[0:,0:,0:,1])
#maxT=np.amax(x_test[0:,0:,0:,2])

x_test[0:,0:,0:,0]=x_test[0:,0:,0:,0]/maxD
x_test[0:,0:,0:,1]=x_test[0:,0:,0:,1]*0
#x_test[0:,0:,0:,2]=x_test[0:,0:,0:,2]*0

In [ ]:
# prediction 
y_pred_line=model_line.predict(x_test)
y_pred_rampRU=model_rampRU.predict(x_test)
y_pred_rampRD=model_rampRD.predict(x_test)
print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

In [ ]:
from sklearn.metrics import confusion_matrix
probability_fitering=np.array([0.999])

#testing the Branch classifier
y_test_line=np.reshape(y_test_line,np.size(y_test_line,0)*np.size(y_test_line,1),'c')
y_pred_line=np.reshape(y_pred_line,np.size(y_pred_line,0)*np.size(y_pred_line,1),'c')

y_pred_line[y_pred_line >= probability_fitering] = 1
y_pred_line[y_pred_line < probability_fitering] = 0
print(confusion_matrix(y_test_line, y_pred_line))

#testing the Ramp Up classifier
y_test_rampRU=np.reshape(y_test_rampRU,np.size(y_test_rampRU,0)*np.size(y_test_rampRU,1),'c')
y_pred_rampRU=np.reshape(y_pred_rampRU,np.size(y_pred_rampRU,0)*np.size(y_pred_rampRU,1),'c')

y_pred_rampRU[y_pred_rampRU >= probability_fitering] = 1
y_pred_rampRU[y_pred_rampRU < probability_fitering] = 0
print(confusion_matrix(y_test_rampRU, y_pred_rampRU))

#testing the Ramp Down classifier
y_test_rampRD=np.reshape(y_test_rampRD,np.size(y_test_rampRD,0)*np.size(y_test_rampRD,1),'c')
y_pred_rampRD=np.reshape(y_pred_rampRD,np.size(y_pred_rampRD,0)*np.size(y_pred_rampRD,1),'c')

y_pred_rampRD[y_pred_rampRD >= probability_fitering] = 1
y_pred_rampRD[y_pred_rampRD < probability_fitering] = 0
print(confusion_matrix(y_test_rampRD, y_pred_rampRD))

print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

In [ ]:
#ouput data reshaping
y_pred_line=np.reshape(y_pred_line,(n_test,Nbranch*12),'c')
y_pred_rampRU=np.reshape(y_pred_rampRU,(n_test,Nunits*11),'c')
y_pred_rampRD=np.reshape(y_pred_rampRD,(n_test,Nunits*11),'c')
print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

#Breaking each scenario into row and columns
y_pred_line=np.reshape(y_pred_line,(n_test,Nbranch,12),'F')
y_pred_rampRU=np.reshape(y_pred_rampRU,(n_test,Nunits,11),'F')
y_pred_rampRD=np.reshape(y_pred_rampRD,(n_test,Nunits,11),'F')
print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

y_pred_line=np.transpose(y_pred_line, (1,2,0))
y_pred_rampRU=np.transpose(y_pred_rampRU, (1,2,0))
y_pred_rampRD=np.transpose(y_pred_rampRD, (1,2,0))
print(y_pred_line.shape)
print(y_pred_rampRU.shape)
print(y_pred_rampRD.shape)

In [ ]:
#saving the predicted result back to image format
predicted_6515 = {"predicted_branch": y_pred_line,"predicted_rampRU": y_pred_rampRU,"predicted_rampRD": y_pred_rampRD}
io.savemat(r'predicted_6515_D.mat', predicted_6515)